Marie Nguyen 

#### LAB 03

In [144]:
# Loading packages
import math
import string
import xml.etree.ElementTree as ET
# Sources: https://docs.python.org/3/library/xml.etree.elementtree.html 

from collections import OrderedDict
# Resource: https://www.geeksforgeeks.org/python-sort-python-dictionaries-by-key-or-value/ 

import nltk
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()


##### Part 1: Implement a tf.idf retrieval algorithm in your code, which uses the weighted sum formula with tf.idf weighting (above) as the similarity function. Set k=2. 

a) Tokenize exactly as you did in the previous assignment.

b) Your inverted lists, in addition to docid, will also need to store tf for each document. 

In [145]:
# Create a linked list 
# Source: https://www.geeksforgeeks.org/python-linked-list/ and Data Structure course

# Initialize a Node class to create a node 
class Node:
    def __init__(self, data):
        self.data = data
        self.next = None

# Initialize a Linkedlist class
class LinkedList:
    def __init__(self):
        self.head = None

    # Add a node to the Linkedlist
    def add(self, data):
        # Initialize a new node with the data 
        new_node = Node(data)
        # If the Linkedlist is empty, assign the new node is the head 
        if not self.head:
            self.head = new_node
            return
        # Otherwise, let the current node is the head 
        current = self.head
        # Loop through the end of the Linkedlist 
        while current.next != None:
            current = current.next
        # Add a new node to the end. 
        current.next = new_node
    
    # Calculate the length of a Linkedlist
    def length(self):
        # Assign current node as the head of the Linkedlist
        current = self.head
        count = 0  # Initilize a variable for counting
        # Interate the the Linkedlist
        while (current != None):
            # Increment the count and move to the next node
            count += 1
            current = current.next
        return count
                  
    def toString(self):
        # Assign a current node as the head 
        current = self.head
        # Initialize an empty string for printing out the linkedlist
        content = " "
        # Loop through all nodes in the linked list
        while (current != None):
            # Add the data of the node to the content 
            content += "    " + str(current.data) + " \n "
            # Move to the next node 
            current = current.next
            
        # content += "None"
        return content

##### Define function for calculating term frequency, given a XML file 

In [146]:
def generate_tf_dictionary(xml_file, stopping, stemming):
    
    # Initialize an array containing stop words:
    with open("stoplist", "r") as stoplistFile:
        stopwords_array = stoplistFile.read().lower().split() #Tokenization
        
    # Initialize a dictionary for the term frequency 
    term_freq_dict = {}
    
    stemmer = PorterStemmer()

    tree = ET.parse(xml_file) 
    root = tree.getroot()
    
    """ Parsing only the headline and text sections of each document
        and save them in a new document """
        
    # Loop through each <DOC> element
    for doc in root.findall("DOC"):
        terms_array = []
        
        # Find the headline and text elements
        headline = doc.find("HEADLINE")
        text_doc = doc.find("TEXT")
        # Find document number element 
        doc_number = doc.find("DOCNO").text
        
        # Remove punctuation and apply lower case
        punctTable = str.maketrans(string.punctuation, " " * len(string.punctuation))
        headline = headline.text.lower().translate(punctTable)
        text_doc = text_doc.text.lower().translate(punctTable)
            
        # Get unique terms from headline in each document 
        for term in headline.split():
            if term not in terms_array:
                # On stemming 
                if stopping == True: 
                    if term not in stopwords_array:
                        terms_array.append(term)
                # On stemming      
                elif stemming == True:
                    terms_array.append(stemmer.stem(term)) #Nomalization
                # Original 
                else:
                    terms_array.append(term)
        
        # Get unique terms from text in each document 
        for term in text_doc.split():
            if term not in terms_array:
                # On stopping 
                if stopping == True: 
                    if term not in stopwords_array:
                        terms_array.append(term)
                # On stemming        
                elif stemming == True:
                    term = stemmer.stem(term) #Nomalization
                    terms_array.append(stemmer.stem(term)) #Nomalization
                # Original     
                else:
                    terms_array.append(term)
        
        # Count each term frequency in the each document
        tf_subdict = {}
        
        # Iterate through the terms array 
        for term in terms_array:
            # If it is a unique term, tf = 1; Otherwise, update tf 
            tf_subdict[term] = tf_subdict.get(term, 0) + 1

        # Update the main term frequency dictionary
        for term in tf_subdict:
            # If the term not in the term frequency dictionary 
            if term not in term_freq_dict:
                # Create an empty sub-dictionary for storing document numbers and related term frequencies. 
                term_freq_dict[term] = {}
            # The value of the document number sub-dictionary is the term frequency  
            term_freq_dict[term][doc_number] = tf_subdict[term]
    # Sorted df dictionary in ascending order
    term_freq_dict = OrderedDict(sorted(term_freq_dict.items()))
    return term_freq_dict

# term_frequencies = generate_tf_dictionary("trec.sample.xml", True, True)
# # Print the results
# for term, doc_freqs in term_frequencies.items():
#     print(f"Term: {term}")
#     for docID, freq in doc_freqs.items():
#         print(f"DocID: {docID}, Term Frequency: {freq}")


Define a function to calculate DOCUMENT FREQUENCY (DF): The number of documents that a term occurs in the collection.


In [147]:
def generate_df_dictionary(xml_file):
    # Initialize a dictionary for document frequency of a term 
    document_freq_dict = {}
    
    # Call the generate_tf_dictionary to generate a term frequency dictionary
    term_freq_dict = generate_tf_dictionary(xml_file, True, True)
    
    df = 0 # Initilize a value to store the document frequency
    
    # Iterate over each term (the key) in the term frequency dictionary:
    for term in term_freq_dict:
        """ Inside the term frequency dictionary, the value for the "term" key is a subdictionary in which 
         its keys are document numbers that the term occurs and the value is the term frequency in that document numbers"""
        # The df (document frequency) is the number of documents the term occurs in the collection
        # The df equals to the length of subdictionary. 
        df = len(term_freq_dict[term])
        
        # Assign the document frequency of the term to the document frequency dictionary:
        document_freq_dict[term] = df

    return document_freq_dict
  
# doc_freqs = generate_df_dictionary("trec.sample.xml")
# for term, df in doc_freqs.items():
#     print(f"term: {term}, df: {df}")

In [148]:
# Define a functionn for getting the list of document ID from the collection 
def all_docIDs(input_XML):
    tree = ET.parse(input_XML) 
    root = tree.getroot()
    
    docIDs_array = LinkedList()
    
    # Loop through each <DOC> element
    for doc in root.findall("DOC"):
        # Find document number element 
        doc_number = doc.find("DOCNO") 
        # Add that document number to the all documents IDs array
        docIDs_array.add(doc_number.text)
        
    return docIDs_array

# Compute inverse document frequencies (IDF) for each unique term in the collection
def compute_idf(df_dict):
    # Initialize an empty array for idf value of a term 
    idf_dict = {}
    # Get the total number of document in the collection
    total_number_docs = all_docIDs("trec.sample.xml").length()
    
    # Iterate through the document frequency array 
    for term in df_dict:
        # Get the df
        df = df_dict[term]
        # Compute idf of each term based on the total number of documents and the df related to the term 
        idf_dict[term] = math.log(total_number_docs / df, 10)
        
    return idf_dict

# print(all_docIDs("trec.sample.xml").length())
# doc_frequency = generate_df_dictionary("trec.sample.xml")
# idf = compute_idf(doc_frequency)

# for term, idf in idf.items():
#     print(f"term: {term}, idf: {idf}")

c) Store the length of each document to use for similarity calculations.  You’ll also need to store average document length.

In [149]:
# Define a function to store the LENGTH of each document:
def compute_document_length(xml_file, stopping, stemming):
    
    # Initialize an array containing stop words:
    with open("stoplist", "r") as stoplistFile:
        stopwords_array = stoplistFile.read().lower().split() #Tokenization
        
    stemmer = PorterStemmer()

    tree = ET.parse(xml_file) 
    root = tree.getroot()
    
    # Initilizr a dictionary for storing the length of each document
    doc_length_dict = {}
    
    for doc in root.findall("DOC"):
        terms_array = []
        
        # Find the headline and text elements
        headline = doc.find("HEADLINE")
        text_doc = doc.find("TEXT")
        # Find document number element 
        doc_number = doc.find("DOCNO").text
        
        # Remove punctuation and apply lower case
        punctTable = str.maketrans(string.punctuation, " " * len(string.punctuation))
        headline = headline.text.lower().translate(punctTable)
        text_doc = text_doc.text.lower().translate(punctTable)
            
        # Get unique terms from headline in each document 
        for term in headline.split():
            if term not in terms_array:
                # On stemming 
                if stopping == True: 
                    if term not in stopwords_array:
                        terms_array.append(term)
                # On stemming      
                elif stemming == True:
                    terms_array.append(stemmer.stem(term)) #Nomalization
                # Original 
                else:
                    terms_array.append(term)
        
        # Get unique terms from text in each document 
        for term in text_doc.split():
            if term not in terms_array:
                # On stopping 
                if stopping == True: 
                    if term not in stopwords_array:
                        terms_array.append(term)
                # On stemming        
                elif stemming == True:
                    term = stemmer.stem(term) #Nomalization
                    terms_array.append(stemmer.stem(term)) #Nomalization
                # Original     
                else:
                    terms_array.append(term)
                    
        doc_length_dict[doc_number] = len(terms_array)
        
    return doc_length_dict
     
# doc_length_dict = compute_document_length("trec.sample.xml", True, True)
# for doc, length in doc_length_dict.items():
#     print(f"doc: {doc}, D: {length}")
    

#### Part 2: Define a Similarity Function For tf.idf calculations:

In [150]:
# Initialize an array containing stop words:
with open("stoplist", "r") as stoplistFile:
    stopwords_array = stoplistFile.read().lower().split() #Tokenization

# Define a SIMILARITY FUNCITON 
def similarity_score(query, term_freq_dict, idf_dict, doc_length_dict, stopping, stemming):
    
    # Preprocessing the query with either stopping or stemming on:
    punctTable = str.maketrans("","",string.punctuation)  # Remove punctuation 
    
    stemmer = PorterStemmer() 
    
    terms = query.strip().translate(punctTable).lower().split() # Preprocessing
    
    query_array = []
    
    for term in terms:
        if term not in query_array:
        # On stopping 
            if stopping == True: 
                if term not in stopwords_array:
                    query_array.append(term)
        # On stemming        
            elif stemming == True:
                query_array.append(stemmer.stem(term)) #Nomalization
        # Original     
            else:
                query_array.append(term)
    
    total_doc_length = 0
    # Initilize a distionary for similarity scores
    tfidf_score = {}
    
    # Call function to get the totall number of documents in the array 
    total_number_docs = all_docIDs("trec.sample.xml").length()
    
    # Iterate through length of each doc in the doc length dictionary 
    for document in doc_length_dict:
        # Sum up the length of all documents 
        total_doc_length += doc_length_dict[document]
    
    # Compute average documenent length
    avg_doc_length = total_doc_length / total_number_docs
    
    # Calculate the similarity between the query and the documents
    # Iterate each terms in the query:
    for term in query_array:
        # Iterate document number (the key of the subdictionary inside the tf dictionary) that the term belongs to:
        for doc_number in term_freq_dict[term].keys():
            # Initalize the tf.idf score for a term in each document. 
            score = 0 
            # Get term frequency of documemnt's term  
            tf_document = term_freq_dict[term][doc_number]
            
            # Get the length of the document 
            document_length = doc_length_dict[doc_number]
            
            # Compute tf * idf score without document length normalization 
            # score = tf_document * idf_dict[term]
            
            # Compute tf * idf score with document length normalization
            score = (tf_document / (tf_document + (2 * document_length / avg_doc_length))) * idf_dict[term]
            
            # Assign score as the value of the key, document number, in the tfidf_score dictionary
            tfidf_score[doc_number] = tfidf_score.get(doc_number,0) + score
    
    # Sort the dictionary by its values in descending order
    tfidf_score = sorted(tfidf_score.items(), key=lambda x:x[1], reverse=True)   
        
    return tfidf_score
    

#### Part 4: Run the search three times

a)	no stopping or stemming, save the results in tfidf.results

b)	stemming only, save the results in tfidf.stem.results

c)	stemming and stopping, save the results in tfidf.all.results

In [151]:
# Process each query:
tfidf_results = {}

with open("queries.lab3.txt", 'r') as file:
    for line in file:
        words = line.strip().lower().split()
        query_id = int(words[0])
        query_terms = ' '.join(words[1:])
        
        term_freq_dict = generate_tf_dictionary("trec.sample.xml", True, True)
        
        doc_freq = generate_df_dictionary("trec.sample.xml")
        idf_dict = compute_idf(doc_freq)
        
        doc_length_dict = compute_document_length("trec.sample.xml", True, True)
        
        # Calculate the score
        score_dict = similarity_score(query_terms, term_freq_dict, idf_dict, doc_length_dict, True, True)
        
        # Store the results for the query after sorting by score
        tfidf_results[query_id] = score_dict

    with open("tfidf.all.results.txt", "w") as input_file:
        # Output the results sorted by query id
        for query_id in range(1, 11): 
            for doc_id, score in tfidf_results.get(query_id, []):
                input_file.writelines(f"{query_id},{doc_id},{score} \n")
        

#### Part 5: Write up a lab report that includes the following:

a) Details on how you implemented the inverted index in project 2. 

In project 2, I used the Element Tree (ET) library for parsing the XML file, extracting document numbers, headlines, and text content. And, a data structure, LinkedList, is utilized to keep track of document IDs in which a term occurs. With an application of ElementTree (ET) library, an input XML file containing multiple documents is represented as a tree. The root of the tree is retrieved. Each document is represented by <DOC> element, which has a document ID, headline, and text content. Looping through the file to find all <DOC> element under the root of the XML file; the document ID, headlines, and text are extracted and nomalized (removing punctuation and lowercasing). Stopping and stemming are also considered on or off. Then, a terms_array is initilized to add unique terms. A inverted dictionary named as term_doc_dict dictionary is initialized to contain the unique terms as keys and a LinkedList of document IDs in which that terms occurs in the collection. For each unique term in the terms_array, if it already exists in term_doc_dict dictionary, the document ID is add into the LinkedList document IDs belonging to that term. If a unique term in a document ID does not appear in the term_doc_dict dictionary, a new linkedlist is generated and the document ID in which that unique term occurs is also added into the just created LinkedList document IDs.  



b) Details on how you implemented the search functions (word overlap and Boolean from proj 2), ranked retrieval using the similarity function above.

In my project 2, I implemented the Boolean retrieval function, using an expression tree. In order to carry out an expression tree from a given query, a TreeNode class is defined, representing each node in a tree and storing data which could be a term or a logical expression later on. In addition, each parent tree node has pointers to its left and right children node. Based on the orignal human readable query, a postfix expression is build. Logical expressions such as "and", "or", "not" are prioritized using a ranking score system (logical_score = {"not": 3, "or": 2, "and": 1}) so that top nodes of building the tree are classified. In this case, "and" will be likely to be a parent node than "not" and "or", and "or" tends to be a parent node of "not". Looping through each token in the infix expression array, if it is a term, add it to the posfix stack, if ut is a logical operators, add it to the logic stack. When the logical operator is "not", it is added to the logic stack. When the logical operators are "and", "or", while the logic stack is not empty and the top element of the stack is not "not", if the top element of the stack has a higher or equal score than the current token, pop the top logical operator from logic stack and appends it to the postfix expression to ensure token with higher score are evaluated first. Then add the current token to the logic stack. And, while there is still logical operators in the logic stack, remove (.pop) it from the stack and add it to the postfix stack. Consequently, from the postfix stack for an expression tree is built in which parent nodes represent logical operators and chil nodes represent terms. 

For building an expression tree, an empty stack is initialized and logical expressions or logical operators are "not", "or", "and". Each token in the postfix array created above is iterated. If a current token is a term, create a term tree node and push that term into the stack. If a current token is an logical operator, pop 2 values from the stack and make the 2 values become children nodes of the logical operator node and push that current node into the stack. If the logical operator is "not", get the top element from the stack, create a "not" tree node and then assign the term as the right node of the "not" parent node. Otherwise, if token is either "and", "or", get the 2 top elements from the stack, create a tree node for either "and" or "or", and then assign the 2 consecutive terms as the right and left children nodes. Afterwards, add that logical operator tree node into the stack. When the first element of the stack is returned, a root node associated with a built logical expression tree is returned. 

Then 3 boolean search functions such as Boolean_search_or (merging the union of 2 LinkedLists document IDs, so if a doument ID is in either list, it is included in the result posting LinkedList), Boolean_search_and (merging the intersection of 2 LinkedLists document IDs, so only document IDs present in the 2 LinkedLists are included in the result posting LinkedList), and Boolean_search_not (generating a list of document IDs that the given term does not represent by subtracting the LinkedLists document ID of a term from the LinkedList all documents IDs in the collection). Finally, the boolean retrieval function is generated to evaluate the expression tree in which the leaf nodes are terms corresponding to LinkedLists document IDs. Starting from the root, each subtree are recursively evaluated based on the operator at each parent node (and, or, not) and combine their results by calling the 3 defined boolean search functions. 

For retrieval using the similarity function above, a term frequency dictionary is generated given an XML file in order to calculate the number of times a term occurs in a document. An application of the Element Tree (ET) library is also used for parsing the XML file, looping through each document element in the tree and finding all headline, text elements and document number. The text is also normalized like removing punctuation and lowcasing; stemming and stopping are also applied for later results comparision.  Then, a terms_array is initilized to add unique terms from text and headline parts. Each term frequency is counted in each document and updated in the tf_subdict dictionary. Iterating each term key in the tf_subdict, if the term is not in the general term frequency dictionary (term_freq_dict), an empty sub-dictionary for storing document numbers and related term frequencies is created. And, the term frequency is assigned as the value of the document number key in which the term occurs in the sub-dictionary. The term in the general term frequency dictionary (term_freq_dict) is sorted in ascending order. Generally, the term frequency dictionary (term_freq_dict) has unique terms in the collection as keys and their values are sub-dictionaries in which the keys are documents containing that terms and the values are term frequencies. Secondly, a document frequency dictionary is generated using the term frequency dictionary. It calculates how many documents that the term occurs in a collection (in other words, it's the length of the sub-dictionaries of each terms). 

Thirdly, with df dictionary and all document IDs dictionary, a function to compute inverse document frequencies (idf) for each unique term in the collection is defined. In the loop of each term in the df dictionary, a document frequency value is retrieved and the log based 10 according to the division between a total number of documents (N) and the df is calculated. Idf is a measure of how much information the word provides whether it's common or rare across a collection of all documents. Fourthly, a compute_document_length function calculates the length of each document in terms of the number of unique terms after preprocessing. Consequently, the similarity_score function takes a query and computes its similarity score with each document using a normalized TF-IDF scores and document normalization. It considers the term frequency, document frequency, and the length of documents to provide a relevance score for each document; stemming and stopping are also considered True or False for results comparision. 

Finally, the queries were read from the queries.lab3.txt file. Each query is pre-processed similarly to document pre-processing (e.g. if stemming is applied in documents, it is also applied in the query, likewise stopping). And, the similarity scores between the query and the document IDs or document numbers are stored in an dictionary (tfidf_score dictionary). The document normalization considers document length to prevent bias towards longer documents. Documents are scored based on how terms in the query appear in each document, taking into account the tf.idf of terms and the length of documents. The ranked retrieval results are sorted by the similarity score in descending order for each query, which means documents are ranked from most to least relevant.

c) A brief commentary on the system as a whole and what you learned from implementing it

Word overlap is an approach that a function retrieves all documents in which terms in the query occur. It is an simple algorithm but if it is a common word across a collection such as "the", more documents will be retrieved and the results might be potentially irrelevant and lack of precision. So, the downside of the approach might be not actually considering whether a term is common or rare. 

Boolean Search involves generating an data structure like an expression tree to evaluate logical expression (sentence containing logical operators such as "and", "or", "not") to combine terms and retrieve more relevant documents. It's more precise than the word overlap approach because it retrieves documents that strictly meet the logical criteria. However, for comple queries, it is more difficult to implement an efficient and flexible boolean search and take more time to process. 

Ranked Retrieval (tf.idf) scores the similarity between a query and documents in a collection. It's based on the frequency of query terms in each document and how common these terms are across all documents. These scores are used to rank documents in terms of its relevance to the query. This approach mainly involves calculating term frequencies, document frequencies, tf.idf scores and document normalization. Ranked Retrieval (tf.idf) also gives a weight to each unique terms through scoring and help us distinguish documents containing common terms versus those containing rare terms. That is aligned with more relevance of retrieved results.

d) What challenges you faced when implementing it

One main challenge I encountered when implementing boolean search is that I was stuck with data structure approach is suitable to use for evaluating the boolean expression. I had a LinkedList document IDs of each term but I was struggling how to retrieve a LinkedList document IDs when there are more than 2 terms and boolean operators (such as "and", "or", "not"). I asked Astrid for help and she recommended me to try to use the expression tree. Expresession tree is similar to a binary tree to some extent. I googled and watched some videos related to how to implement an expression tree. So, given a boolean expression query, "and" has a higher rank than "or" and "not", and "or" has a higher rank than "or". "And" is aimed to be the top of the expression tree. A given infix boolean expression is converted into a postfix using a stack. Then, another stack is used to generate an expression tree from this postfix expression. For each term in a query, a term tree node is created and pushed into the stack. And for each logical operator, 2 terms from the stack are poped and make them become children nodes of the logical operator node then push back that sub-tree onto the stack. The expression tree is executed from the bottom to the top in which the leaves are LinkedList document IDs of terms in the query. From all that knowledge, I proceeded to implement the Boolean search. 

Resources: https://www.geeksforgeeks.org/evaluation-of-expression-tree/

https://www.geeksforgeeks.org/convert-infix-expression-to-postfix-expression/

While implementing the similarity scores tf.idf, my challenge was to attempt to understand how to build a term frequency dictionary and how to implement the similarity function, especially with normalization document length. I went to office hours and I understood the concept better. Documents containing query terms have a score of 0 at first, then a similarity score between the query and the document is calculated and added to the documents' scores. A dictionary of documents in which query terms occur and their associated similarity scores with the query are generated. It also took me some time to figure out how to implement stemming and stopping into the code and compare results with my classmates.  

e) Any ideas on how to improve and scale your implementation (e.g. if you were to index a much larger collection, would your implementation run efficiently? If not, how could you improve it?)

If I were to index a much larger collection, it would be take a much longer running time for text pre-processing as well as implementing Boolean Search and Ranked Retrieval (tf.idf with document normalization) approaches. As a consequence, my implementation would not run efficiently. To improve it, I could index based on common phrases and rare phrases to optimize quick search and attempt to apply n-grams to improve the relevance and performance of the retrieval system. In addition, I could take into account for partitioning the inverted index across a collection. 

f) Compare and contrast the results for the three runs.  For example, what do you notice about them, what do you think explains the differences between them, which ones appear to be the best? 

In terms of running time, only stemming tooks longer time, around 5 minutes minutes, and a similarity score between a document and a query associated with applying only stemming is smaller compared to without stemming and stopping and with both stemming and stopping. The similarity scores without stopping and stemming is higher than the similarity scores with both stemming and stopping, indicating that stop words are common words in the documents. Applying stemming and stopping results a lower similarity scores but the results might be more relevant. 

For picking the best model, it depends on the context of the document. If the context contains lots of common words and those words are mentioned in the query, maybe not using stopping could be helpful. In the example 1 as below, not applying stemming and stopping appears to have the highest similarity score between the query and the document. While with both stemming and stopping the similarity score goes down, indicating that reducing words might loose some information although it helps reduce irrelevant common words such as "the" and retrieve more relevant documents associated with terms telling the context of the document. With only stemming, the similarity score even goes down, showing that stemming words withough removing stop words is not an efficient way and the short form of the words might not be relavant enough to capture the context of the document.  

Example 1: 

1,3829,1.6252003432981004 (without stemming and stopping)

1,3829,1.3768646193254095 (with both stemming and stopping)

1,3829,1.3260318143528464 (with only stemming)

Example 2:

1,3599,1.139406065211134 (without stemming and stopping)

1,3599,1.2140747543651442 (with both stemming and stopping)

1,3599,1.1427578663149143 (only stemming)

However, things are different in the second example. The combined approach of stemming and stopping seems to be the most effective for this document, indicating that focusing on terms representing meaning of a document works effeciently. Only stemming brings an slightly improvement compared to not applying stemming and stopping, showing that removing stopwords might help improve the similarity score and even retrieve top relevant documents. In general, the second example illustrates the benefits of stemming and stopping over retaining the original text. 

g) Describe your approach for evaluating and comparing the results.

To compare the results, I created 3 txt files with tf.idf.results (no stopping and stemming), tfidf.stem.results (with stemming only), and tfidf.all.results (stemming and stopping). In the tf.idf.results, the stopping and stemming arguments are False, False, respectively. And, in the tfidf.stem.results, the stopping and stemming arguments are False, True, respectively. Similarly, in the tf.idf.results, the stopping and stemming arguments are True, True, respectively.

The query is preprocessed similarly to the document. Stemming applied in the documents is also used in the query. Likewise, stopping applied in the documents is also applied in the query. That ensures the terms in the query and documets are consistent. Different scores getting from the 3 approaches are written in the 3 abovementioned txt files and I used the search function to find a specific document and then compare their 3 results of similar scores manually. My approach for evaluating and comparing the results is not effcient.